This is the Glob-style path to the subfolder containing niftis of interest
- For example, from the base_directory, */tissue_segment_z_scores will look for all subjects, all session folders within subjects, and then check the tissue_segment_z_scores folder. 


In [1]:
base_directory = r'/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORNIX/compound_vtas'
shared_glob_pattern = r'*.nii*'

In [2]:
from calvin_utils.file_utils.import_functions import GiiNiiFileImport
dv_df = GiiNiiFileImport(import_path=base_directory, file_pattern=shared_glob_pattern).run()
dv_df

Attempting to import from: /Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORNIX/compound_vtas/*.nii*


,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORNIX/compound_vtas/142_vat_compound_bl.nii,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORNIX/compound_vtas/113_vat_compound_bl.nii,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORNIX/compound_vtas/143_vat_compound_bl.nii,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORNIX/compound_vtas/112_vat_compound_bl.nii,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORNIX/compound_vtas/108_vat_compound_bl.nii,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORNIX/compound_vtas/111_vat_compound_bl.nii,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORNIX/compound_vtas/140_vat_compound_bl.nii,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORNIX/compound_vtas/109_vat_compound_bl.nii,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORNIX/compound_vtas/110_vat_compound_bl.nii,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORNIX/compound_vtas/141_vat_compound_bl.nii,...,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORNIX/compound_vtas/125_vat_compound_bl.nii,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORNIX/compound_vtas/124_vat_compound_bl.nii,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORNIX/compound_vtas/127_vat_compound_bl.nii,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORNIX/compound_vtas/126_vat_compound_bl.nii,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORNIX/compound_vtas/139_vat_compound_bl.nii,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORNIX/compound_vtas/120_vat_compound_bl.nii,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORNIX/compound_vtas/138_vat_compound_bl.nii,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORNIX/compound_vtas/121_vat_compound_bl.nii,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORNIX/compound_vtas/122_vat_compound_bl.nii,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORNIX/compound_vtas/123_vat_compound_bl.nii
0,0.0,0.0,0.0,0.0,-32768.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,-32768.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,-32768.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,-32768.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,-32768.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902624,0.0,0.0,0.0,0.0,-32768.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902625,0.0,0.0,0.0,0.0,-32768.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902626,0.0,0.0,0.0,0.0,-32768.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902627,0.0,0.0,0.0,0.0,-32768.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**Extract Subject ID From File Names**
Using the example filenames that have been printed above, please define a general string:
1) Preceding the subject ID.
2) Proceeding the subject ID. 

This Should Often Be Left Default

In [3]:
# preceding_id = '/sub-'
# proceeding_id = ''

In [4]:
# from calvin_utils.file_utils.import_functions import GiiNiiFileImport
# dv_df = GiiNiiFileImport.splice_colnames(dv_df, preceding_id, proceeding_id)
# dv_df

# Load Map to Check Damage Within

Import Region of Interest Masks

In [5]:
base_directory = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/ccm_memory/results/notebook_07/best_maps/weighted_average'
shared_glob_pattern = 'all_datasets_weighted_avg_r_map_pcc_inverse_r_map.nii.gz'

In [6]:
from calvin_utils.file_utils.import_functions import GiiNiiFileImport
iv_df = GiiNiiFileImport(import_path=base_directory, file_pattern=shared_glob_pattern).run()
iv_df

Attempting to import from: /Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/ccm_memory/results/notebook_07/best_maps/weighted_average/all_datasets_weighted_avg_r_map_pcc_inverse_r_map.nii.gz


,/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/ccm_memory/results/notebook_07/best_maps/weighted_average/all_datasets_weighted_avg_r_map_pcc_inverse_r_map.nii.gz
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
902624,0.0
902625,0.0
902626,0.0
902627,0.0


Extract Damage Scores Per Region of Interest

In [7]:
mask_path = '/Users/cu135/hires_backdrops/MNI/MNI152_T1_2mm_brain_mask.nii'

In [8]:
from calvin_utils.nifti_utils.damage_score_utils import DamageScorer
damage_scorer = DamageScorer(mask_path, dv_df, iv_df)
dmg_df = damage_scorer.calculate_damage_scores()
dmg_df = damage_scorer.sort_dataframes_by_index(dmg_df)
dmg_df

,all_datasets_weighted_avg_r_map_pcc_inverse_r_map_spatial_corr,all_datasets_weighted_avg_r_map_pcc_inverse_r_map_cosine,all_datasets_weighted_avg_r_map_pcc_inverse_r_map_sum,all_datasets_weighted_avg_r_map_pcc_inverse_r_map_average_subject_in_target,all_datasets_weighted_avg_r_map_pcc_inverse_r_map_average_target_in_subject,all_datasets_weighted_avg_r_map_pcc_inverse_r_map_num_in_roi
path,,,,,,
/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORNIX/compound_vtas/142_vat_compound_bl.nii,-0.018661,-0.018730,-1.605878e+01,-1.204023e-04,-0.411764,0.0
/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORNIX/compound_vtas/113_vat_compound_bl.nii,-0.017585,-0.017657,-1.535560e+01,-1.151301e-04,-0.374527,0.0
/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORNIX/compound_vtas/143_vat_compound_bl.nii,-0.015800,-0.015867,-1.294785e+01,-9.707781e-05,-0.349942,0.0
/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORNIX/compound_vtas/112_vat_compound_bl.nii,-0.009666,-0.009730,-7.452506e+00,-5.587592e-05,-0.232891,0.0
/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORNIX/compound_vtas/108_vat_compound_bl.nii,0.000000,0.011304,5.262041e+07,3.945269e+02,0.000000,0.0
/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORNIX/compound_vtas/111_vat_compound_bl.nii,-0.006765,-0.006822,-4.740761e+00,-3.554433e-05,-0.182337,0.0
/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORNIX/compound_vtas/140_vat_compound_bl.nii,-0.011191,-0.011252,-8.452113e+00,-6.337057e-05,-0.281737,0.0
/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORNIX/compound_vtas/109_vat_compound_bl.nii,-0.007280,-0.007306,-2.436905e+00,-1.827094e-05,-0.487381,0.0
/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORNIX/compound_vtas/110_vat_compound_bl.nii,-0.013665,-0.013725,-1.016500e+01,-7.621314e-05,-0.350517,0.0


Save the Results

In [9]:
analysis = ''
session = 'dbs'
root_dir = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/BIDS_AD_DBS_FORNIX/study_metadata/study_metadata/ccm_study/metadata.csv'

In [10]:
# damage_scorer.save_csv_to_metadata(dmg_df, root_dir=root_dir, analysis=analysis, ses=session, dry_run=False)

Save Results into Master CSV

In [11]:
master_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/BIDS_AD_DBS_FORNIX/study_metadata/study_metadata/ccm_study/metadata.csv'
sheet = None

In [12]:
from calvin_utils.permutation_analysis_utils.statsmodels_palm import CalvinStatsmodelsPalm
cal_palm = CalvinStatsmodelsPalm(input_csv_path=master_path, output_dir=None, sheet=sheet)
# Call the process_nifti_paths method
data_df = cal_palm.read_and_display_data()
data_df

,Dataset,Subject,path,conn_path_v,roi_path,conn_path,age,adas_q4_improve,age_at_surgery,sex,...,train_prune_test_performs_highly_pcc_inverse_r_map_sum,train_prune_test_performs_highly_pcc_inverse_r_map_average_subject_in_target,train_prune_test_performs_highly_pcc_inverse_r_map_average_target_in_subject,train_prune_test_performs_highly_pcc_inverse_r_map_num_in_roi,all_datasets_unweighted_avg_r_map_pcc_inverse_r_map_spatial_corr,all_datasets_unweighted_avg_r_map_pcc_inverse_r_map_cosine,all_datasets_unweighted_avg_r_map_pcc_inverse_r_map_sum,all_datasets_unweighted_avg_r_map_pcc_inverse_r_map_average_subject_in_target,all_datasets_unweighted_avg_r_map_pcc_inverse_r_map_average_target_in_subject,all_datasets_unweighted_avg_r_map_pcc_inverse_r_map_num_in_roi
0,Advance_AD_DBS_FORNIX,101,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORN...,/Volumes/Expansion/datasets/AD_dataset/derivat...,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORN...,/Volumes/OneDrive/datasets/AD_dataset/derivati...,62,-1.0,age_at_surgery,m,...,-4.639712e+00,-3.830000e-05,-0.193321,0,0.007275,0.007161,7.629485e+00,5.643235e-05,0.317895,0.0
1,Advance_AD_DBS_FORNIX,102,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORN...,/Volumes/Expansion/datasets/AD_dataset/derivat...,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORN...,/Volumes/OneDrive/datasets/AD_dataset/derivati...,77,0.0,age_at_surgery,m,...,-1.892006e-02,-1.560000e-07,-0.018920,0,0.001737,0.001713,2.953667e-02,2.184713e-07,0.029537,0.0
2,Advance_AD_DBS_FORNIX,103,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORN...,/Volumes/Expansion/datasets/AD_dataset/derivat...,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORN...,/Volumes/OneDrive/datasets/AD_dataset/derivati...,76,2.0,age_at_surgery,m,...,-2.946343e+00,-2.430000e-05,-0.226642,0,0.005465,0.005383,4.133908e+00,3.057692e-05,0.317993,0.0
3,Advance_AD_DBS_FORNIX,104,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORN...,/Volumes/Expansion/datasets/AD_dataset/derivat...,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORN...,/Volumes/OneDrive/datasets/AD_dataset/derivati...,65,0.0,age_at_surgery,m,...,-1.040313e+01,-8.590000e-05,-0.192651,0,0.009785,0.009618,1.493415e+01,1.104621e-04,0.276558,0.0
4,Advance_AD_DBS_FORNIX,105,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORN...,/Volumes/Expansion/datasets/AD_dataset/derivat...,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORN...,/Volumes/OneDrive/datasets/AD_dataset/derivati...,50,NaN,age_at_surgery,m,...,-4.523702e+00,-3.730000e-05,-0.141366,0,0.004498,0.004375,5.545394e+00,4.101714e-05,0.173294,0.0
5,Advance_AD_DBS_FORNIX,106,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORN...,/Volumes/Expansion/datasets/AD_dataset/derivat...,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORN...,/Volumes/OneDrive/datasets/AD_dataset/derivati...,66,1.0,age_at_surgery,m,...,-5.908583e+00,-4.880000e-05,-0.236343,0,0.007240,0.007121,8.245272e+00,6.098710e-05,0.329811,0.0
6,Advance_AD_DBS_FORNIX,107,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORN...,/Volumes/Expansion/datasets/AD_dataset/derivat...,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORN...,/Volumes/OneDrive/datasets/AD_dataset/derivati...,64,NaN,age_at_surgery,m,...,-7.998152e+00,-6.600000e-05,-0.173873,0,0.006540,0.006381,9.803634e+00,7.251370e-05,0.213122,0.0
7,Advance_AD_DBS_FORNIX,108,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORN...,/Volumes/Expansion/datasets/AD_dataset/derivat...,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORN...,/Volumes/OneDrive/datasets/AD_dataset/derivati...,60,NaN,age_at_surgery,m,...,1.605915e+08,1.325319e+03,0.000000,0,0.000000,0.022200,1.603924e+08,1.186361e+03,0.000000,0.0
8,Advance_AD_DBS_FORNIX,109,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORN...,/Volumes/Expansion/datasets/AD_dataset/derivat...,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORN...,/Volumes/OneDrive/datasets/AD_dataset/derivati...,72,2.0,age_at_surgery,m,...,-1.190919e+00,-9.830000e-06,-0.238184,0,0.003797,0.003744,1.938138e+00,1.433566e-05,0.387628,0.0
9,Advance_AD_DBS_FORNIX,110,/Volumes/OneTouch/datasets/ADVA

In [13]:
# Perform an outer join on dmg_df and data_df using the 'path' column
merged_df = data_df.merge(dmg_df, on='path', how='outer')
merged_df

,Dataset,Subject,path,conn_path_v,roi_path,conn_path,age,adas_q4_improve,age_at_surgery,sex,...,all_datasets_unweighted_avg_r_map_pcc_inverse_r_map_sum,all_datasets_unweighted_avg_r_map_pcc_inverse_r_map_average_subject_in_target,all_datasets_unweighted_avg_r_map_pcc_inverse_r_map_average_target_in_subject,all_datasets_unweighted_avg_r_map_pcc_inverse_r_map_num_in_roi,all_datasets_weighted_avg_r_map_pcc_inverse_r_map_spatial_corr,all_datasets_weighted_avg_r_map_pcc_inverse_r_map_cosine,all_datasets_weighted_avg_r_map_pcc_inverse_r_map_sum,all_datasets_weighted_avg_r_map_pcc_inverse_r_map_average_subject_in_target,all_datasets_weighted_avg_r_map_pcc_inverse_r_map_average_target_in_subject,all_datasets_weighted_avg_r_map_pcc_inverse_r_map_num_in_roi
0,Advance_AD_DBS_FORNIX,101,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORN...,/Volumes/Expansion/datasets/AD_dataset/derivat...,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORN...,/Volumes/OneDrive/datasets/AD_dataset/derivati...,62,-1.0,age_at_surgery,m,...,7.629485e+00,5.643235e-05,0.317895,0.0,-0.015906,-0.015961,-1.095586e+01,-8.214265e-05,-0.456494,0.0
1,Advance_AD_DBS_FORNIX,102,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORN...,/Volumes/Expansion/datasets/AD_dataset/derivat...,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORN...,/Volumes/OneDrive/datasets/AD_dataset/derivati...,77,0.0,age_at_surgery,m,...,2.953667e-02,2.184713e-07,0.029537,0.0,-0.003831,-0.003843,-4.269790e-02,-3.201318e-07,-0.042698,0.0
2,Advance_AD_DBS_FORNIX,103,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORN...,/Volumes/Expansion/datasets/AD_dataset/derivat...,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORN...,/Volumes/OneDrive/datasets/AD_dataset/derivati...,76,2.0,age_at_surgery,m,...,4.133908e+00,3.057692e-05,0.317993,0.0,-0.011980,-0.012020,-5.946663e+00,-4.458570e-05,-0.457436,0.0
3,Advance_AD_DBS_FORNIX,104,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORN...,/Volumes/Expansion/datasets/AD_dataset/derivat...,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORN...,/Volumes/OneDrive/datasets/AD_dataset/derivati...,65,0.0,age_at_surgery,m,...,1.493415e+01,1.104621e-04,0.276558,0.0,-0.023138,-0.023220,-2.322964e+01,-1.741666e-04,-0.430179,0.0
4,Advance_AD_DBS_FORNIX,105,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORN...,/Volumes/Expansion/datasets/AD_dataset/derivat...,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORN...,/Volumes/OneDrive/datasets/AD_dataset/derivati...,50,NaN,age_at_surgery,m,...,5.545394e+00,4.101714e-05,0.173294,0.0,-0.010258,-0.010319,-8.427158e+00,-6.318347e-05,-0.263349,0.0
5,Advance_AD_DBS_FORNIX,106,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORN...,/Volumes/Expansion/datasets/AD_dataset/derivat...,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORN...,/Volumes/OneDrive/datasets/AD_dataset/derivati...,66,1.0,age_at_surgery,m,...,8.245272e+00,6.098710e-05,0.329811,0.0,-0.016914,-0.016972,-1.266152e+01,-9.493104e-05,-0.506461,0.0
6,Advance_AD_DBS_FORNIX,107,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORN...,/Volumes/Expansion/datasets/AD_dataset/derivat...,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORN...,/Volumes/OneDrive/datasets/AD_dataset/derivati...,64,NaN,age_at_surgery,m,...,9.803634e+00,7.251370e-05,0.213122,0.0,-0.013800,-0.013879,-1.373754e+01,-1.029986e-04,-0.298642,0.0
7,Advance_AD_DBS_FORNIX,108,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORN...,/Volumes/Expansion/datasets/AD_dataset/derivat...,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORN...,/Volumes/OneDrive/datasets/AD_dataset/derivati...,60,NaN,age_at_surgery,m,...,1.603924e+08,1.186361e+03,0.000000,0.0,0.000000,0.011304,5.262041e+07,3.945269e+02,0.000000,0.0
8,Advance_AD_DBS_FORNIX,109,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORN...,/Volumes/Expansion/datasets/AD_dataset/derivat...,/Volumes/OneTouch/datasets/ADVANCE_AD_DBS_FORN...,/Volumes/OneDrive/datasets/AD_dataset/derivati...,72,2.0,age_at_surgery,m,...,1.938138e+00,1.433566e-05,0.387628,0.0,-0.007280,-0.007306,-2.436905e+00,-1.827094e-05,-0.487381,0.0
9,Advance_AD_DBS_FORNIX,110,/Volu

In [14]:
merged_df.to_csv(master_path, index=False)

All done. Enjoy your analyses. 

--Calvin 